In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import h5py
from time import time
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
import pandas as pd
import labrotation.two_photon_session as tps
import seaborn as sns
import uuid  # for unique labeling of sessions and coupling arrays (mouse velocity, distance, ...) to sessions in dataframe 
from matplotlib import cm  # colormap
import datadoc_util
from labrotation import two_photon_session as tps
from datetime import datetime
import seaborn as sns
from math import floor

In [ ]:
sns.set(font_scale=2)
sns.set_style("whitegrid")

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
save_dsets = False

In [ ]:
save_figs = False
save_as_eps = False
save_as_pdf = False
if save_as_eps:
    output_format = ".eps"
elif save_as_pdf:
    output_format=".pdf"
else:
    output_format = ".jpg"
if save_figs:
    print(output_format)

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

## Load all seizures dataset

In [ ]:
df_events = ddoc.getEventsDf()
df_events = df_events[df_events["event_type"] == "sz"] 

In [ ]:
event_traces_fpath = fh.open_file("Open .h5 file containing assembled traces for all seizures!")
print(event_traces_fpath)

In [ ]:
df_colors = ddoc.getColorings()

In [ ]:
df_colors

In [ ]:
traces_ca1 = []
traces_nc = []

mouse_ids_ca1 = []
mouse_ids_nc = []

uuids_ca1 = []
uuids_nc = []

session_uuids_ca1 = []
session_uuids_nc = []

recording_break_points_ca1 = []
recording_break_points_nc = []

n_bl_frames = 5000
n_am_frames = 5000

# first keys are event uuids, inside the following dataset names:
# 'lfp_mov_t', 'lfp_mov_y', 'lfp_t', 'lfp_y', 'lv_dist', 'lv_rounds', 
# 'lv_running', 'lv_speed', 'lv_t_s', 'lv_totdist', 'mean_fluo'
with h5py.File(event_traces_fpath, "r") as hf:
    for uuid in hf.keys():
        win_type = hf[uuid].attrs["window_type"]
        mean_fluo = np.array(hf[uuid]["mean_fluo"])
        assert n_bl_frames == hf[uuid].attrs["n_bl_frames"]
        assert n_am_frames == hf[uuid].attrs["n_am_frames"]
        mouse_id = hf[uuid].attrs["mouse_id"]
        if win_type == "Cx":
            traces_nc.append(mean_fluo)
            uuids_nc.append(uuid)
            session_uuids_nc.append(hf[uuid].attrs["session_uuids"])
            recording_break_points_nc.append(hf[uuid].attrs["recording_break_points"])
            mouse_ids_nc.append(mouse_id)
        elif win_type == "CA1":
            traces_ca1.append(mean_fluo)
            uuids_ca1.append(uuid)
            session_uuids_ca1.append(hf[uuid].attrs["session_uuids"])
            recording_break_points_ca1.append(hf[uuid].attrs["recording_break_points"])
            mouse_ids_ca1.append(mouse_id)
        else:
            print(f"{win_type} not recognized window type")

In [ ]:
colors_ca1 = [df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0] for mouse_id in mouse_ids_ca1]
colors_nc = [df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0] for mouse_id in mouse_ids_nc]

## Get baseline values
Calculated as lowest 5% of data points in baseline segment. 

In [ ]:
lowest_percent = 0.05  # 5% of baseline to be used 

In [ ]:
# FIXME: these take lowest values of the whole traces!
#baselines_ca1 = [np.sort(traces_ca1[i][:floor(lowest_percent*n_bl_frames)]) for i in range(len(traces_ca1))]
#baselines_nc = [np.sort(traces_nc[i][:floor(lowest_percent*n_bl_frames)]) for i in range(len(traces_nc))]

baselines_ca1 = [np.min(traces_ca1[i][:n_bl_frames]) for i in range(len(traces_ca1))]
baselines_nc = [np.min(traces_nc[i][:n_bl_frames]) for i in range(len(traces_nc))]

In [ ]:
# amplitude between bl and sz ampl, sd ampl!

## Get aftermath values
in 20 sec windows, get minimum value of fluorescence

### Calculate first normal frames
Use data documentation for corresponding recording

In [ ]:
# ca1: need to find first segment after the "sd_extinction" segment, and find the corresponding index in the (5000 + sz + 5000) traces
first_frames_ca1 = []
rec_uuids_ca1 = []
for i_event in range(len(traces_ca1)):
    event_uuid = uuids_ca1[i_event]
    # get all segments belonging to aftermath
    df_event = df_events[(df_events["event_uuid"] == event_uuid) & (df_events["interval_type"] == "am")]
    # for all recordings contributing to aftermath, look which one contains sd_extinction
    i_frame = len(traces_ca1[i_event]) - n_am_frames  # points to first am frame right now
    next_segment_stop = False  # flag to stop on reaching next segment
    found_frame = False  # flag to mark if first frame to take was found
    am_rec_uuid = None
    for i_row, am_row in df_event.iterrows():  # loop over recordings participating in aftermath trace
        # begin and end frames of am in current recording
        am_begin_frame = am_row["begin_frame"]
        am_end_frame = am_row["end_frame"]
        # uuid of current recording
        rec_uuid = am_row["recording_uuid"]
        # get all segments after start of am
        i_first_am = ddoc.getSegmentForFrame(rec_uuid, am_begin_frame).index[0]
        i_last_am = ddoc.getSegmentForFrame(rec_uuid, am_end_frame).index[0]
        am_segments = ddoc.getSegmentsForUUID(rec_uuid).loc[i_first_am:i_last_am+1]
        am_rec_uuid = rec_uuid
        for i_segment_row, segment_row in am_segments.iterrows():
            if next_segment_stop:  # first segment after sd_extinction reached. Take this as start for baseline return observation
                break
            if segment_row["interval_type"] == "sd_extinction":
                next_segment_stop = True
            segment_length = segment_row["frame_end"] - segment_row["frame_begin"] + 1  # both inclusive -> need +1
            i_frame += segment_length
        if found_frame:
            break
    first_frames_ca1.append(i_frame)
    rec_uuids_ca1.append(am_rec_uuid)

# nc: there is no SD, so just take first am frame as it is
first_frames_nc = [len(traces_nc[i]) - n_am_frames for i in range(len(traces_nc))]


In [ ]:
interval_length_seconds = 10
interval_length = 15*interval_length_seconds  # 15 Hz * 20 seconds
n_intervals = 11

In [ ]:
aftermath_ca1 = [ np.array([np.min( traces_ca1[i][ first_frames_ca1[i] + j*interval_length : first_frames_ca1[i] + (j+1)*interval_length  ] )  for j in range(n_intervals)]) for i in range(len(traces_ca1)) ] 
aftermath_nc = [ np.array([np.min( traces_nc[i][ first_frames_nc[i] + j*interval_length : first_frames_nc[i] + (j+1)*interval_length  ] )  for j in range(n_intervals)]) for i in range(len(traces_nc)) ]

## Create dataframe
Columns should be: uuid, value (numeric), value_type (bl, 20s, 40s, ... 200 s)

In [ ]:
#col_names = ["baseline_mean", "baseline_std"] + [f"{20*i}s" for i in range(1, n_intervals+1)]
data_dict = {"uuid": [], "value": [], "value_type": []}

# get baseline values for CA1 and NC

for i_event_ca1 in range(len(baselines_ca1)):
    uuids = [uuids_ca1[i_event_ca1]]  # only one baseline value per event
    value_types = ["bl"]
    data_dict["uuid"] += uuids
    data_dict["value"] += [baselines_ca1[i_event_ca1]]
    data_dict["value_type"] += value_types

for i_event_nc in range(len(baselines_nc)):
    uuids = [uuids_nc[i_event_nc]] # only one baseline value per event
    value_types = ["bl"]
    data_dict["uuid"] += uuids
    data_dict["value"] += [baselines_nc[i_event_nc]]
    data_dict["value_type"] += value_types        

# get 20, 40, ..., 200 s values for CA1 and NC

for i_event_ca1 in range(len(aftermath_ca1)):
    uuids = [uuids_ca1[i_event_ca1]]*len(aftermath_ca1[i_event_ca1])
    value_types = [f"{(i+1)*interval_length_seconds}s" for i in range(n_intervals)]
    assert len(uuids) == len(value_types)
    assert len(uuids) == len(aftermath_ca1[i_event_ca1])
    data_dict["uuid"] += uuids
    data_dict["value"] += list(aftermath_ca1[i_event_ca1])
    data_dict["value_type"] += value_types

for i_event_nc in range(len(aftermath_nc)):
    uuids = [uuids_nc[i_event_nc]]*len(aftermath_nc[i_event_nc])
    value_types = [f"{(i+1)*interval_length_seconds}s" for i in range(n_intervals)]
    assert len(uuids) == len(value_types)
    assert len(uuids) == len(aftermath_nc[i_event_nc])
    data_dict["uuid"] += uuids
    data_dict["value"] += list(aftermath_nc[i_event_nc])
    data_dict["value_type"] += value_types   
     

In [ ]:
df = pd.DataFrame(data=data_dict)

In [ ]:
fig = plt.figure(figsize=(18,18))
sns.lineplot(data=df, palette="tab10", x="value_type", y="value", hue="uuid", linewidth=2.5, legend=False)
plt.show()

In [ ]:
all_bl_traces = []
all_am_traces = []
for i_tr in range(len(traces_ca1)):
    all_am_traces.append(traces_ca1[i_tr][first_frames_ca1[i_tr]:])
    all_bl_traces.append(traces_ca1[i_tr][n_bl_frames - 1000 :n_bl_frames])
for i_tr in range(len(traces_nc)):
    all_am_traces.append( traces_nc[i_tr][first_frames_nc[i_tr]:])
    all_bl_traces.append(traces_nc[i_tr][n_bl_frames - 1000 :n_bl_frames])
    
bl_x = np.array([i-len(all_bl_traces[0])+1 for i in range(len(all_bl_traces[0]))])

In [ ]:
# TODO: lowess filter? Somehow filter this signal!
fig = plt.figure(figsize=(18,18))
for tr in all_bl_traces:
    plt.plot(bl_x, tr)
for tr in all_am_traces:
    plt.plot(tr)
plt.ylim((0, 60))
#plt.xlim((-10,10))
plt.show()

# Recovery analysis

In [ ]:
# get a list of trace indices that are sorted by mouse
event_uuid_mouse_id_i_trace_ca1 = []  # list of (event_uuid, mouse_id, i_trace) tuples
event_uuid_mouse_id_i_trace_nc  = []  # list of (event_uuid, mouse_id, i_trace) tuples


for event_uuid in df_events["event_uuid"].unique():
    mouse_id = df_events[df_events["event_uuid"] == event_uuid].mouse_id.iloc[0]
    if event_uuid in uuids_ca1:
        i_trace = uuids_ca1.index(event_uuid)
        event_uuid_mouse_id_i_trace_ca1.append((event_uuid, mouse_id, i_trace))
    elif event_uuid in uuids_nc:
        i_trace = uuids_nc.index(event_uuid)
        event_uuid_mouse_id_i_trace_nc.append((event_uuid, mouse_id, i_trace))
        
    else:
        print(f"Unknown event_uuid: {event_uuid}")

In [ ]:
window_width_s = 10
window_step_s = 5
imaging_frequency = 15. # in Hz
n_frames_before_nc = 200  # include 200 frames just before aftermath for NC recordings  
n_frames_before_ca1 = 0
n_windows_post_darkest = 40 # dataset consists of bl, darkest point, and this many windows post darkest point

# define baseline windows
bl_windows_nc = [(3950, 4100) for i in range(len(traces_nc))]  # for neocortex, allow for ~1 min before Sz (LFP sz comes earlier)
bl_windows_ca1 = [(4850, 5000) for i in range(len(traces_ca1))]  # for CA1, immediately before Sz onset

i_frame_begin_bl = 3850  # in 0-indexing, the first frame to be included in baseline
i_frame_end_bl = 4000  # in 0-indexing, the first frame after baseline (i.e. not included)

time_points = ["bl", "darkest"] + [f"{(i+1)*window_step_s}s" for i in range(n_windows_post_darkest)]
time_points_numeric = [-1, 0] + [(i+1)*window_step_s for i in range(n_windows_post_darkest)]
time_points_numeric = np.array(time_points_numeric)

window_width_frames = int(window_width_s*imaging_frequency)
window_step_frames = int(window_step_s*imaging_frequency)

def get_metric_for_window(trace_window):
    lowest_5p_indices = np.argsort(trace_window)[:int(0.05*len(trace_window))]
    lowest_5p = trace_window[lowest_5p_indices]
    return np.median(lowest_5p)

def get_recovery_data(complete_trace, i_frame_begin_bl, i_frame_end_bl, n_frames_before_am=0):
    # n_frames_before_am: for NC, need to include a few frames before the segment "aftermath" begins, due to mistakes in 
    # manual classification. In CA1, this is not necessary
    
    metrics_list = []
    x_list = []
    
    
    # The complete trace should consist of 5000 bl, x Sz, 5000 am frames.
    # get bl as just before Sz begin
    bl_trace = complete_trace[i_frame_begin_bl:i_frame_end_bl]
    x_bl = (i_frame_begin_bl + i_frame_end_bl)//2  # TODO: assign proper x
    y_bl = get_metric_for_window(bl_trace)
    
    # add bl to dataset
    x_list.append(x_bl)
    metrics_list.append(y_bl)
    
    # get am 5p darkest points
    sorted_indices = np.argsort(complete_trace)
    sorted_am_indices = sorted_indices[sorted_indices > len(complete_trace) - 5000 - n_frames_before_am]
    am_x_5p_lowest = sorted_am_indices[:int(0.05*(5000+n_frames_before_am))] 
    
    # get single coordinate for darkest part
    # find darkest 5p, take earliest 50 of them, get median frame index of these, round down to integer frame
    x_am_darkest = int(floor(np.median(np.sort(am_x_5p_lowest)[:50])))
    
    # create sliding windows, calculate metric
    for i_window in range(n_windows_post_darkest+1):  # window around darkest point + n_windows_post_darkest windows
        x_val = x_am_darkest + i_window*window_step_frames
        window_half_width = window_width_frames//2
        window_trace = complete_trace[x_val - window_half_width : x_val + window_half_width]
        y_val = get_metric_for_window(window_trace)
        
        x_list.append(x_val)
        metrics_list.append(y_val)
        
    return (x_list, metrics_list)
    

## Calculate metrics for all data

In [ ]:
x_recovery_ca1 = [[] for i in range(len(traces_ca1))]
y_recovery_ca1 = [[] for i in range(len(traces_ca1))]

x_recovery_nc = [[] for i in range(len(traces_nc))]
y_recovery_nc = [[] for i in range(len(traces_nc))]


for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    x_data, y_data = get_recovery_data(traces_nc[i_trace], bl_windows_nc[i_trace][0], bl_windows_nc[i_trace][1], n_frames_before_nc)
    x_recovery_nc[i_trace] = x_data
    y_recovery_nc[i_trace] = y_data

    
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    x_data, y_data = get_recovery_data(traces_ca1[i_trace], bl_windows_ca1[i_trace][0], bl_windows_ca1[i_trace][1], n_frames_before_ca1)
    x_recovery_ca1[i_trace] = x_data
    y_recovery_ca1[i_trace] = y_data
    
x_recovery_ca1 = np.array(x_recovery_ca1, dtype=np.int16)
x_recovery_nc = np.array(x_recovery_nc, dtype=np.int16)

y_recovery_ca1 = np.array(y_recovery_ca1,)
y_recovery_nc = np.array(y_recovery_nc, )


## Get time points of recovery

In [ ]:
i_recovery_nc = []
i_recovery_ca1 = []

recovery_ratio = 0.95  # reach 99% of baseline to be considered recovered

def get_recovery_index(recovery_trace):
    baseline = recovery_trace[0]  # bl, darkest, windows...
    if np.max(recovery_trace[2:]) < recovery_ratio*baseline:
        return -1
    i_recovery = np.argmax(recovery_trace[2:] >= recovery_ratio*baseline) + 2  # shift back to index in whole trace
    return i_recovery

for recovery_trace in y_recovery_nc:
    i_recovery = get_recovery_index(recovery_trace)
    i_recovery_nc.append(i_recovery)
    
for recovery_trace in y_recovery_ca1:
    i_recovery = get_recovery_index(recovery_trace)
    i_recovery_ca1.append(i_recovery)
    
    

In [ ]:
time_points_numeric[i_recovery_nc]

## Plot results

## CA1

In [ ]:
fig = plt.figure(figsize=(18,18))
amplitude = 100.
offset = 0.

for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(amplitude*(traces_ca1[i_trace] - min(traces_ca1[i_trace]))/(max(traces_ca1[i_trace] - min(traces_ca1[i_trace])))+offset, color=c )
    plt.vlines(x=x_recovery_ca1[i_trace], ymin=offset-0.1*amplitude, ymax=offset+1.1*amplitude)
    for window_center in x_recovery_ca1[i_trace]:
        plt.hlines(y=offset-5, xmin=window_center-window_width_frames//2, xmax= window_center+window_width_frames//2, color="red")
    offset += 1.3*amplitude
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"ca1_recovery_overview_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(24,18))
x_ticks = [i for i in range(len(y_recovery_ca1[0]))]
offset = 0.0
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(y_recovery_ca1[i_trace] - y_recovery_ca1[i_trace][0] + offset, "o-", color=c, markersize=10 )
    plt.hlines(y=offset, xmin=0, xmax=len(y_recovery_ca1[i_trace]), color="red")
    plt.vlines(ymin=offset-5, ymax=offset+5, x=i_recovery_ca1[i_trace], linewidth=5, color=c)
    offset += 10
plt.xticks(x_ticks, time_points)
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"ca1_recovery_time_points_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

## NC

In [ ]:
fig = plt.figure(figsize=(18,18))
amplitude = 100.
offset = 0.

for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(amplitude*(traces_nc[i_trace] - min(traces_nc[i_trace]))/(max(traces_nc[i_trace] - min(traces_nc[i_trace])))+offset, color=c )
    plt.vlines(x=x_recovery_nc[i_trace], ymin=offset-0.1*amplitude, ymax=offset+1.1*amplitude)
    for window_center in x_recovery_nc[i_trace]:
        plt.hlines(y=offset-5, xmin=window_center-window_width_frames//2, xmax= window_center+window_width_frames//2, color="red")
    offset += 1.3*amplitude
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"nc_recovery_overview_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
offset = 0.0
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(y_recovery_nc[i_trace] - y_recovery_nc[i_trace][0] + offset, "o-", color=c, markersize=10 )
    plt.hlines(y=offset, xmin=0, xmax=len(y_recovery_nc[i_trace]), color="red")
    plt.vlines(ymin=offset-5, ymax=offset+5, x=i_recovery_nc[i_trace], linewidth=5, color=c)
    offset += 10
plt.xticks(x_ticks, time_points)
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"nc_recovery_time_points_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

### Baseline - darkest difference

In [ ]:
bl_minus_darkest_ca1 = y_recovery_ca1[:,0] - y_recovery_ca1[:,1]
bl_minus_darkest_nc = y_recovery_nc[:,0] - y_recovery_nc[:,1]

In [ ]:
df_bl_minus_darkest_ca1 = pd.DataFrame(data=bl_minus_darkest_ca1, columns=["bl-darkest"])
df_bl_minus_darkest_ca1["window_type"] = "ca1"

df_bl_minus_darkest_nc = pd.DataFrame(data=bl_minus_darkest_nc, columns=["bl-darkest"])
df_bl_minus_darkest_nc["window_type"] = "nc"

df_bl_minus_darkest_combined = pd.concat([df_bl_minus_darkest_ca1, df_bl_minus_darkest_nc]) 

In [ ]:
fig, axs=plt.subplots(1,2,figsize=(8,6), sharey=True)
left= sns.barplot(data=df_bl_minus_darkest_combined, x="window_type", y="bl-darkest", ax=axs[0])
right = sns.stripplot(data=df_bl_minus_darkest_combined, x="window_type", y="bl-darkest", hue="window_type", legend=False, s=15, ax=axs[1])


plt.show()

## Create dataframes

In [ ]:
# add np.nan for not found onset values
time_points_numeric_extended = np.concatenate([time_points_numeric, np.array([np.nan])])

In [ ]:
mouse_ids_nc = ["" for i in range(len(i_recovery_nc))]
event_uuids_nc = ["" for i in range(len(i_recovery_nc))]
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    mouse_ids_nc[i_trace] = mouse_id
    event_uuids_nc[i_trace] = event_uuid
df_recovery_nc = pd.DataFrame(columns=["i_recovery"],data=i_recovery_nc)
df_recovery_nc["t_recovery"] = time_points_numeric_extended[i_recovery_nc]
df_recovery_nc["event_uuid"] =  event_uuids_nc
df_recovery_nc["mouse_id"] = mouse_ids_nc
df_recovery_nc["window_type"] = "nc"
#df_recovery_nc = df_recovery_nc.sort_values(by="mouse_id").reset_index().drop(columns=["index"])

In [ ]:
mouse_ids_ca1 = ["" for i in range(len(i_recovery_ca1))]
event_uuids_ca1 = ["" for i in range(len(i_recovery_ca1))]
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    mouse_ids_ca1[i_trace] = mouse_id
    event_uuids_ca1[i_trace] = event_uuid
df_recovery_ca1 = pd.DataFrame(columns=["i_recovery"],data=i_recovery_ca1)
df_recovery_ca1["t_recovery"] = time_points_numeric_extended[i_recovery_ca1]
df_recovery_ca1["event_uuid"] =  event_uuids_ca1
df_recovery_ca1["mouse_id"] = mouse_ids_ca1
df_recovery_ca1["window_type"] = "ca1"
#df_recovery_ca1 = df_recovery_nc.sort_values(by="mouse_id").reset_index().drop(columns=["index"])

In [ ]:
df_recovery_combined = pd.concat([df_recovery_ca1, df_recovery_nc])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8,6), sharey=True)
left = sns.barplot(x="window_type", y="t_recovery", data=df_recovery_combined, ax= axs[0])
right = sns.stripplot(x="window_type", y="t_recovery", hue="window_type", data=df_recovery_combined.dropna(), ax=axs[1], s=15, legend=False)
left.set(ylim=(0, 200))
#right.set(ylim=(0, 200))

plt.show()

## Export datasets

In [ ]:
if save_dsets:
    fpath_df_recovery_nc = os.path.join(output_folder, f"bl_recovery_nc_{get_datetime_for_fname()}.xlsx")
    fpath_df_recovery_ca1 = os.path.join(output_folder, f"bl_recovery_ca1_{get_datetime_for_fname()}.xlsx")
    df_recovery_nc.to_excel(fpath_df_recovery_nc, index=False)
    print(f"Saved NC data to {fpath_df_recovery_nc}")
    df_recovery_ca1.to_excel(fpath_df_recovery_ca1, index=False)
    print(f"Saved NC data to {fpath_df_recovery_ca1}")
    
    fpath_recovery_bl_minus_darkest_slope = os.path.join(output_folder, f"bl_recovery_bl-minus-darkest_slope-darkest-2min_{get_datetime_for_fname()}.xlsx")
    df_recovery_combined[["mouse_id", "window_type", "bl-darkest", "slope_darkest_to_2min"]].to_excel(fpath_recovery_bl_minus_darkest_slope, index=False)
    print(f"Saved recovery baseline - darkest and slope darkest to 2min post to {fpath_recovery_bl_minus_darkest_slope}")
    
    
    

In [ ]:
df_recovery_combined

## Compare bl-darkest in CA1 vs NC

In [ ]:
fig = plt.figure(figsize=(6,10))
ax = plt.gca()
sns.stripplot(data=df_recovery_combined, x="window_type", y="bl-darkest", hue="window_type", s=20,  ax=ax, legend=None)
sns.barplot(data=df_recovery_combined, x="window_type", y="bl-darkest", ax=ax, alpha=0.5)
plt.tight_layout()
if save_figs:
    savefig_fpath = os.path.join(output_folder, f"recovery_bl-minus-darkest_{get_datetime_for_fname()}{output_format}")
    plt.savefig(savefig_fpath)
    print(f"Saved as {savefig_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,10))
ax = plt.gca()
sns.violinplot(data=df_recovery_combined, x="window_type", y="bl-darkest", ax=ax, inner="quart")

plt.show()

### Test significance of difference
t-test assumes normal distribution, which probably is not the case...

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
ttest_ind(df_recovery_combined[df_recovery_combined["window_type"] == "ca1"]["bl-darkest"], df_recovery_combined[df_recovery_combined["window_type"] == "nc"]["bl-darkest"], equal_var=False)

## Compare recovery rates CA1 vs NC

In [ ]:
fig = plt.figure(figsize=(6,10))
ax = plt.gca()
sns.barplot(data=df_recovery_combined.dropna(), x="window_type", y="slope_darkest_to_2min", errorbar="sd",ax=ax, alpha=0.5)
sns.stripplot(data=df_recovery_combined.dropna(), x="window_type", y="slope_darkest_to_2min", hue="window_type", s=20, legend=None)
plt.tight_layout()
if save_figs:
    savefig_fpath = os.path.join(output_folder, f"recovery_slope_darkest-to-2minpost_{get_datetime_for_fname()}{output_format}")
    plt.savefig(savefig_fpath)
    print(f"Saved as {savefig_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,10))

sns.violinplot(data=df_recovery_combined.dropna(), x="window_type", y="slope_darkest_to_2min", inner="quart")

plt.show()

# Simple amplitude comparison

In [ ]:
# calculate baseline value as follows: take n_bl_window_frames frames before Sz; get 5% lowest values, take mean of these.
n_bl_window_frames = 30*15  # 30 s (*15 Hz) frames to look at to get baseline fluorescence
lowest_percent_bl_window = 0.05  # lowest 5%
amplitudes_ca1 = []
for i in range(len(traces_ca1)):
    bl_val_ca1 = np.sort(traces_ca1[i][n_bl_frames-n_bl_window_frames+1:n_bl_frames])[:int(lowest_percent_bl_window*n_bl_window_frames)].mean()  # take mean of lowest 5% of baseline values
    len_sz = len(traces_ca1[i]) -  n_am_frames - n_bl_frames
    sz_amp_val_ca1 = np.flip(np.sort(traces_ca1[i][n_bl_frames:n_bl_frames+len_sz+1]))[:int(0.05*len_sz)].mean()  # take mean of highest 5% of sz values
    sd_amp_val_ca1 = np.flip(np.sort(traces_ca1[i][n_bl_frames+len_sz+1:]))[:int(0.05*n_am_frames)].mean()
    amplitudes_ca1.append([bl_val_ca1, sz_amp_val_ca1, sd_amp_val_ca1])
amplitudes_ca1 = np.array(amplitudes_ca1)

In [ ]:
amplitudes_nc = []
for i in range(len(traces_nc)):
    bl_val_nc = np.sort(traces_nc[i][:n_bl_frames])[:int(0.5*n_bl_frames)].mean()  # take mean of lowest 5% of baseline values
    len_sz = len(traces_nc[i]) -  n_am_frames - n_bl_frames
    sz_amp_val_nc = np.flip(np.sort(traces_nc[i][n_bl_frames:n_bl_frames+len_sz+1]))[:int(0.05*len_sz)].mean()  # take mean of highest 5% of sz values
    amplitudes_nc.append([bl_val_nc, sz_amp_val_nc])
amplitudes_nc = np.array(amplitudes_nc)

In [ ]:
fig = plt.figure(figsize=(18,12))
for i_sz in range(len(amplitudes_ca1)):
    plt.plot(amplitudes_ca1[i_sz], color=colors_ca1[i_sz])
plt.show()

In [ ]:
# check SD category was recognized properly
fig = plt.figure()
for i in range(len(traces_ca1)):
    len_sz = len(traces_ca1[i]) -  n_am_frames - n_bl_frames
    plt.plot(traces_ca1[i][n_bl_frames+len_sz+1:], color=colors_ca1[i])
plt.show()

In [ ]:
amplitude_ratios_ca1 = []  # [i][0] is sz:bl ratio, [i][1] is sd:sz
for i, amplitudes in enumerate(amplitudes_ca1):
    ratio_sz_bl = amplitudes[1]/amplitudes[0]
    ratio_sd_sz = amplitudes[2]/amplitudes[1]
    amplitude_ratios_ca1.append((mouse_ids_ca1[i], uuids_ca1[i], "CA1", ratio_sz_bl, ratio_sd_sz))
df_ratios_ca1 = pd.DataFrame(data=amplitude_ratios_ca1, columns=["mouse_id", "sz_uuid", "win_type", "sz:bl", "sd:sz"])

In [ ]:
amplitude_ratios_nc = []  # [i][0] is sz:bl ratio, [i][1] is sd:sz
for i, amplitudes in enumerate(amplitudes_nc):
    ratio_sz_bl = amplitudes[1]/amplitudes[0]
    amplitude_ratios_nc.append((mouse_ids_nc[i], uuids_nc[i], "NC", ratio_sz_bl))
df_ratios_nc = pd.DataFrame(data=amplitude_ratios_nc, columns=["mouse_id", "sz_uuid", "win_type", "sz:bl"])

In [ ]:
df_ratios = pd.concat([df_ratios_ca1, df_ratios_nc])

In [ ]:
df_ratios_long = pd.melt(df_ratios, id_vars=["sz_uuid", "mouse_id", "win_type"], value_vars=["sz:bl", "sd:sz"], var_name = ["ratio_type"] )

In [ ]:
df_ratios_long[df_ratios_long["ratio_type"] == "sz:bl"]

In [ ]:
fig = plt.figure(figsize=(8,14))
for i_sz in range(len(amplitudes_ca1)):
    plt.plot(amplitudes_ca1[i_sz], "o-", color=colors_ca1[i_sz], )
for i_sz_nc in range(len(amplitudes_nc)):
    plt.plot(amplitudes_nc[i_sz_nc], "o-", color=colors_nc[i_sz_nc])
plt.ylabel("fluorescence (a.u.)")
    
plt.vlines([0,1,2], 0, 500)
    
plt.text(0.05, 450, "BL", fontdict=None)
plt.text(1.05, 450, "Sz", fontdict=None)
plt.text(1.8, 450, "SD", fontdict=None)

frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.xaxis.grid(False)
#frame1.axes.yaxis.set_ticklabels([])
if save_figs:
    output_fpath = os.path.join(output_folder, f"amplitudes_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
amplitudes_ca1_to_bl = amplitudes_ca1[:,1:] - amplitudes_ca1[:,0, None]
amplitudes_nc_to_bl = amplitudes_nc[:,1:] - amplitudes_nc[:,0, None]

In [ ]:
fig = plt.figure(figsize=(4,10))
for i_sz in range(len(amplitudes_ca1_to_bl)):
    plt.plot(amplitudes_ca1_to_bl[i_sz], "o-", color=colors_ca1[i_sz], )
#for i_sz_nc in range(len(amplitudes_nc_to_bl)):
#    plt.plot(amplitudes_nc_to_bl[i_sz_nc], "o-", color=colors_nc[i_sz_nc])
plt.ylabel("fluorescence (a.u.)")
    
plt.vlines([0,1], 0, 500)
    
plt.text(0.05, 450, "Sz-bl", fontdict=None)
plt.text(0.68, 450, "SD-bl", fontdict=None)

frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.xaxis.grid(False)
#frame1.axes.yaxis.set_ticklabels([])
if save_figs:
    output_fpath = os.path.join(output_folder, f"amplitudes_to_bl_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
from scipy.stats import ttest_rel

In [ ]:
ttest_result = ttest_rel(amplitudes_ca1_to_bl.T[1][:-1], amplitudes_ca1_to_bl.T[0][:-1])

In [ ]:
ttest_result

In [ ]:
df_amplitudes_ca1 = pd.DataFrame(columns=["Sz-bl", "SD-bl"], data=amplitudes_ca1_to_bl)

In [ ]:
save_to_xlsx = False
if save_to_xlsx:
    amplitudes_export_fpath_ca1 = f"D:\\Downloads\\amplitudes_to_bl_ca1_{get_datetime_for_fname()}.xlsx"
    df_amplitudes_ca1.to_excel(amplitudes_export_fpath_ca1, index=False)
    print(f"Saved file to {amplitudes_export_fpath_ca1}")
    